In [1]:
# @title
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def volume_profile_fixed_range(
    df_ohlcv: pd.DataFrame,
    range_start=None, range_end=None,
    price_bins=60,
    ticker="TICKER",
    show_ema=False, ema_span=100
):
    """
    df_ohlcv: DataFrame con columnas ['Open','High','Low','Close','Volume'] e índice datetime
    range_start, range_end: 'YYYY-MM-DD' o Timestamp para limitar el rango fijo
    price_bins: número de niveles de precio para el histograma
    """

    df = df_ohlcv.copy()
    # Filtro por rango fijo (si se pasa)
    if range_start is not None:
        df = df[df.index >= pd.to_datetime(range_start)]
    if range_end is not None:
        df = df[df.index <= pd.to_datetime(range_end)]
    if df.empty:
        raise ValueError("No hay datos en el rango seleccionado.")

    # Rango de precios en el período
    pmin = float(df["Low"].min())
    pmax = float(df["High"].max())
    # Bordes de bins
    edges = np.linspace(pmin, pmax, price_bins + 1)
    centers = 0.5 * (edges[:-1] + edges[1:])

    # Acumulador de volumen por bin (inicializado en 0)
    vp = np.zeros(price_bins, dtype=float)

    # Reparto proporcional del volumen de cada vela a los bins que cruza
    # (asumiendo distribución uniforme dentro del rango High-Low)
    for _, row in df.iterrows():
        lo, hi, vol = float(row["Low"]), float(row["High"]), float(row["Volume"])
        if vol <= 0 or hi <= lo:
            continue

        # Encuentra bins que intersectan [lo, hi]
        # Por eficiencia, localizamos índices aproximados
        i0 = np.searchsorted(edges, lo, side="right") - 1
        i1 = np.searchsorted(edges, hi, side="left")
        i0 = max(i0, 0)
        i1 = min(i1, price_bins - 1)

        total_range = hi - lo
        for i in range(i0, i1 + 1):
            # Segmento del bin que cae dentro de [lo, hi]
            seg_lo = max(lo, edges[i])
            seg_hi = min(hi, edges[i + 1])
            if seg_hi > seg_lo:
                frac = (seg_hi - seg_lo) / total_range
                vp[i] += vol * frac

    # Normaliza para mostrar longitudes razonables (opcional)
    vp_norm = vp / (vp.max() if vp.max() > 0 else 1.0)

    # ====== FIGURA ======
    fig = go.Figure()

    # 1) Perfil de volumen (barras horizontales a la izquierda)
    fig.add_trace(go.Bar(
        x=vp_norm,              # longitud (normalizada 0-1)
        y=centers,              # eje de precio
        orientation='h',
        name='Volume Profile (norm)',
        hovertemplate="Precio: %{y:.4f}<br>Vol: %{customdata:.0f}<extra></extra>",
        customdata=vp.reshape(-1,1),
        marker=dict(opacity=0.4),
        xaxis="x2",  # usa eje x secundario (panel izquierdo)
        yaxis="y"    # comparte eje de precios
    ))

    # 2) Velas
    fig.add_trace(go.Candlestick(
        x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"],
        name=ticker, yaxis="y", xaxis="x"
    ))

    # 3) EMA opcional
    if show_ema:
        ema = df["Close"].ewm(span=ema_span, adjust=False).mean()
        fig.add_trace(go.Scatter(
            x=df.index, y=ema, mode="lines",
            name=f"EMA{ema_span}", line=dict(width=2), yaxis="y", xaxis="x"
        ))

    # ====== LAYOUT CON DOS PANELES (izq: perfil, der: velas) ======
    fig.update_layout(
        template="plotly_white",
        height=720,
        title=f"{ticker} — Volume Profile (Fixed Range)  |  {range_start} → {range_end}",
        showlegend=True,
        # Definimos dominios: x2 (perfil) ocupa el 22% izquierdo, x (velas) el resto
        xaxis2=dict(domain=[0.0, 0.22], anchor="y", title="Intensidad volumen (norm)", showgrid=False),
        xaxis=dict(domain=[0.27, 1.0], anchor="y", title="Fecha"),
        # Eje de precios compartido
        yaxis=dict(title="Precio", side="right"),
        bargap=0,
        hovermode="x unified"
    )

    # Ejes sin zoom en el panel del perfil si quieres “fixedrange”
    # (el usuario puede seguir haciendo zoom sobre el panel principal)
    fig.update_xaxes(fixedrange=True, matches=None, row=None, col=None, selector=dict(_deprecated=False), secondary_y=False, xaxis="x2")
    # Mantener el eje de precios compartido libre para zoom
    #fig.show()
    return fig

# === Ejemplo de uso ===
# Asumiendo que 'data' ya tiene columnas ['Open','High','Low','Close','Volume'] e índice datetime
# volume_profile_fixed_range(data, range_start="2025-03-01", range_end="2025-07-31", price_bins=70,
#                            ticker="DOGE-USD", show_ema=True, ema_span=100)

In [2]:
import yfinance as yf

ticker = "DOGE-USD"
data = yf.download(ticker, start="2024-01-01", end="2025-09-01", auto_adjust=False)

# droplavel columns
data.columns = data.columns.droplevel(1)
data.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2024-01-01,0.092024,0.092024,0.092046,0.088838,0.089473,298687049
2024-01-02,0.091204,0.091204,0.094330,0.090804,0.092037,496309010
2024-01-03,0.082042,0.082042,0.092093,0.081018,0.091205,1026941029
2024-01-04,0.084131,0.084131,0.084655,0.081551,0.082049,514230239
2024-01-05,0.082773,0.082773,0.084755,0.081235,0.084121,547983037


In [3]:
volume_profile_fixed_range(
    data,
    range_start="2025-03-01",   # fecha inicial del rango fijo
    range_end="2025-07-31",     # fecha final del rango fijo
    price_bins=70,              # número de niveles de precio (más bins = más detalle)
    ticker="DOGE-USD",          # etiqueta para el gráfico
    show_ema=True,              # mostrar EMA opcional
    ema_span=100                # periodo de la EMA
)